Analysis

In [15]:
import pandas as pd 
import re

pd.set_option('display.max_colwidth', None)

In [16]:
df1 = pd.read_csv("/workspaces/erp_programme_updates/data/programme_updates_topics.csv")
df2 = pd.read_csv("/workspaces/erp_programme_updates/data/updates_with_project_people.csv")

In [17]:
#combine these two df
df = pd.merge(df1, df2, on="id", how="left")

In [18]:
df.head()

,id,text_x,text_preprocessed,topic_id,topic_label,newsletter_number,date,link,domain,organisation,text_y,project,key_person
0,237938bb-bac2-4977-8b83-b12f02c37478,"Poorer pupils do worse at school – here's how to reduce the attainment gap Stephen Gorard writes for The Conversation on closing the attainment gap, specifically by looking at 'spread(ing) out the most disadvantaged students between schools' and better calibrating the funding to follow the most disadvantaged pupils.",poor pupil bad school here attainment gap conversation attainment gap disadvantaged student school funding disadvantaged pupil,3,Children’s Rights & Participation,2,2023-07-16,https://theconversation.com/poorer-pupils-do-worse-at-school-heres-how-to-reduce-the-attainment-gap-205535,theconversation.com,conversation,"Poorer pupils do worse at school – here's how to reduce the attainment gap Stephen Gorard writes for The Conversation on closing the attainment gap, specifically by looking at 'spread(ing) out the most disadvantaged students between schools' and better calibrating the funding to follow the most disadvantaged pupils.",Recruitment and retention of ethnic minority teachers,Stephen Gorard
1,e1f9ff04-4b3a-48da-b7db-82a12c2eb364,A three-part series on leadership from Toby Greany and team on TES: Part 1: Headteacher recruitment crisis: 5 tips for action,threepart series leadership team headteacher recruitment crisis tip action,2,Knowledge Exchange & Briefings,2,2023-07-16,https://www.tes.com/magazine/leadership/staff-management/headteacher-recruitment-crisis-applications,www.tes.com,tes,A three-part series on leadership from Toby Greany and team on TES: Part 1: Headteacher recruitment crisis: 5 tips for action,Sustainable school leadership,Toby Greany
2,a414d421-d953-4300-9fdd-f7b8e081912d,"Teacher recruitment, retention and development - rethinking policy and practice priorities seminar – 3 July 2023 Inaugural event in the 'What Matters in education?' series Blog post on the topic - Looking at teacher recruitment and retention in a new light",teacher recruitment retention development policy practice priority seminar july inaugural event education series blog post topic teacher recruitment retention new light,2,Knowledge Exchange & Briefings,3,2023-07-20,https://blogs.ucl.ac.uk/ioe/2023/06/27/looking-at-teacher-recruitment-and-retention-in-a-new-light,blogs.ucl.ac.uk,ucl_blog,"Teacher recruitment, retention and development - rethinking policy and practice priorities seminar – 3 July 2023 Inaugural event in the 'What Matters in education?' series Blog post on the topic - Looking at teacher recruitment and retention in a new light",Education Research Programme,NaN
3,a6c9957e-6707-426c-831e-a5709b4152e3,Teaching for Digital Citizenship Delphi conference View the recording of David Lundie's event (held on May 22). BERA membership required to access. https://www.bera.ac.uk/media/teaching-for-digital-citizenship-may2023,digital citizenship delphi conference view recording lundie event bera membership access,4,Teacher Recruitment & Retention,3,2023-07-20,https://www.bera.ac.uk/media/teaching-for-digital-citizenship-may2023,www.bera.ac.uk,bera,Teaching for Digital Citizenship Delphi conference View the recording of David Lundie's event (held on May 22). BERA membership required to access. https://www.bera.ac.uk/media/teaching-for-digital-citizenship-may2023,Teaching for digital citizenship,David Lundie
4,1d1ad500-8555-4988-8d4a-51a5a3d18566,My corona*: listening to children in corona times Resource shared via Sarah Chicken's website - https://childrens-participation.org/ This research paper brings together three of the young journalists who worked on The Corona Times Journal to reflect on their experiences of being involved in the project. https://www.tandfonline.com/doi/epdf/10.1080/13642987.2022.2061954?needAccess=true&role=button,corona child corona times resource chicken website research paper young journalist corona times journal experience project,6,S

In [19]:
df.columns

Index(['id', 'text_x', 'text_preprocessed', 'topic_id', 'topic_label',
       'newsletter_number', 'date', 'link', 'domain', 'organisation', 'text_y',
       'project', 'key_person'],
      dtype='object')

In [20]:
save_path = "/workspaces/erp_programme_updates/data/final_with_topics_merged.csv"
df.to_csv(save_path, index=False)
print(f"Merged file saved to: {save_path}")

Merged file saved to: /workspaces/erp_programme_updates/data/final_with_topics_merged.csv


# Clean Up 

In [21]:
#renamd "organisation" to "platform" 
df  = df.rename(columns={"organisation": "platform"})

In [22]:
df.columns

Index(['id', 'text_x', 'text_preprocessed', 'topic_id', 'topic_label',
       'newsletter_number', 'date', 'link', 'domain', 'platform', 'text_y',
       'project', 'key_person'],
      dtype='object')

In [23]:
df["text_preprocessed"]

0                                                                                                                                                                                                                    poor pupil bad school here attainment gap conversation attainment gap disadvantaged student school funding disadvantaged pupil
1                                                                                                                                                                                                                                                                        threepart series leadership team headteacher recruitment crisis tip action
2                                                                                                                                                                          teacher recruitment retention development policy practice priority seminar july inaugural event education series blog post topic teacher recruitment 

In [10]:
df.groupby(["project","topic_label"]).size()


project                                                topic_label                       
Cross-project                                          Digital Citizenship & Technology       3
                                                       Knowledge Exchange & Briefings         2
                                                       School Leadership & Sustainability     3
Decentring the 'resilient teacher'                     Digital Citizenship & Technology       1
                                                       Knowledge Exchange & Briefings         1
Digital empowerment in language teaching (DELTEA)      Digital Citizenship & Technology       1
                                                       Teacher Recruitment & Retention        1
Education Research Programme                           Children’s Rights & Participation      1
                                                       Digital Citizenship & Technology       1
                                              

In [24]:
df.columns

Index(['id', 'text_x', 'text_preprocessed', 'topic_id', 'topic_label',
       'newsletter_number', 'date', 'link', 'domain', 'platform', 'text_y',
       'project', 'key_person'],
      dtype='object')

# Classify the output e.g. publication, report, blog, social media etc. 

In [26]:
def classify_output_type(text):
    if not isinstance(text, str):
        return "other"

    text = text.lower()

    categories = {
        "publication": [
            r"\bpublish(ed)?\b", r"\bvolume(ed)?\b", r"\bpublication\b", r"\barticle\b", r"\bpaper\b",
            r"\breport\b", r"\bbriefing\b", r"\bbriefing note\b", r"\bpreprint\b",
            r"\bjournal\b", r"\bchapter\b", r"\bspecial issue\b", r"\bmanuscript\b",
            r"\binternational review\b", r"\bberj\b"           
        ],

        "event": [
            r"\bworkshop\b", r"\bconference\b", r"\bwebinar\b", r"\bseminar\b",
            r"\btalk\b", r"\bpanel\b", r"\broundtable\b", r"\bsymposium\b",
            r"\blecture\b", r"\bpresentation\b", r"\btraining session\b",
            r"\bfestival\b",
            r"\bwhat matters in education\b", 
            r"\bevent\b", 
            r"\bonline and in person\b",
            r"\bspeakers?\b", 
            r"\binteractive workshops?\b",
            r"\bin conversation with\b"
        ],

        "media": [
            r"\bcritics say\b", r"\bpodcast\b", r"\bvideo\b", r"\binterview\b", r"\bpress\b",
            r"\bradio\b", r"\bbroadcast\b", r"\bnews\b",
            r"\bthe conversation\b",  r"\btheconversation\b", r"\bguardian\b", r"\btes\b",
            r"\bschoolsweek\b", r"\bschools week\b", r"\byorkshire post\b", r"\bmagazine\b", r"\bwonkhe\b", 
        ],

        "blog/vlog": [
            r"\bblog\b", r"\bblogpost\b", r"\bblog post\b", r"\bonline article\b",
            r"\bpost\b", r"\bvlog\b"
        ],

        "policy_engagement": [
            r"\bcall for evidence\b", r"\bconsultation\b", r"\bminister\b", r"\bdfe\b",
            r"\bdepartment for education\b", r"\bparliament\b",
            r"\bpolicy\b", r"\bpolicymaker\b", r"\bstakeholder\b",
            r"\bevidence session\b"
        ],

        "research_progress": [
            r"\bdata collection\b", r"\bsurvey\b", r"\bfieldwork\b",
            r"\binterviews?\b", r"\bfocus groups?\b", r"\banalysis\b",
            r"\bcoding\b", r"\btranscription\b",
            r"\binvestigator\b", r"\bdemo\b"
        ],

        "collaboration": [
            r"\bcollaborat", r"\bjoint\b", r"\bcross-project\b",
            r"\bpartnered\b", r"\bworked with\b"
        ],

        "resource_output": [
            r"\btoolkit\b", r"\bresource\b", r"\bguidance\b", r"\bdataset\b",
            r"\btraining material\b", r"\bframework\b"
        ],

        "award_recognition": [
            r"\baward\b", r"\bprize\b", r"\bfunding\b",
            r"\bgrant\b", r"\brecognised\b", r"\bcommended\b"
        ]
    }

    # Search patterns in priority order
    for cat, patterns in categories.items():
        for pattern in patterns:
            if re.search(pattern, text):
                return cat

    return "other"

In [27]:
df["output_type"] = df["text_x"].apply(classify_output_type)

In [28]:
counts = df["output_type"].value_counts(dropna=False)
counts["TOTAL"] = counts.sum()
counts

output_type
event                 45
publication           34
media                 22
blog/vlog             12
research_progress      4
policy_engagement      1
TOTAL                118
Name: count, dtype: int64

In [29]:
df[["text_x", "output_type"]].head(-20)

,text_x,output_type
0,"Poorer pupils do worse at school – here's how to reduce the attainment gap Stephen Gorard writes for The Conversation on closing the attainment gap, specifically by looking at 'spread(ing) out the most disadvantaged students between schools' and better calibrating the funding to follow the most disadvantaged pupils.",media
1,A three-part series on leadership from Toby Greany and team on TES: Part 1: Headteacher recruitment crisis: 5 tips for action,media
2,"Teacher recruitment, retention and development - rethinking policy and practice priorities seminar – 3 July 2023 Inaugural event in the 'What Matters in education?' series Blog post on the topic - Looking at teacher recruitment and retention in a new light",event
3,Teaching for Digital Citizenship Delphi conference View the recording of David Lundie's event (held on May 22). BERA membership required to access. https://www.bera.ac.uk/media/teaching-for-digital-citizenship-may2023,event
4,My corona*: listening to children in corona times Resource shared via Sarah Chicken's website - https://childrens-participation.org/ This research paper brings together three of the young journalists who worked on The Corona Times Journal to reflect on their experiences of being involved in the project. https://www.tandfonline.com/doi/epdf/10.1080/13642987.2022.2061954?needAccess=true&role=button,publication
...,...,...
93,Teach First Lunch and Learn: The five-year-itch: Retaining Early Career Teachers 1 April 11-11.50 Rebecca Snell will be presenting at Teach First's Lunch & Learn on her BERJ paper The five-year-itch: Retaining Early Career Teachers Link to join,publication
94,The ERP's double symposium entitled: ' Doing Policy Relevant Research: Using knowledge mobilisation and knowledge exchange strategies to translate findings into actionable insights.' Part One. Technology in education and Part Two. Teachers and Teaching will be presented at BERA 2025. The symposia will combine insights from seven projects in the programme. The ERP have created a new Knowledge Exchange Resource Hub . This brings together a range of resources we are aware of that are designed to support research engagement with different stakeholders. We welcome other suggestions. Please get in touch if you are aware of other resources that we are missing.,event
95,Children's Participation in Schools Annual Symposium 2025 Catch up with the video of the team's annual symposium featuring talks from some of their teacher participants.,event
96,"19 May 2025, 5:30 pm–7:00 pm. Speakers Toby Greany, Nottingham University Steph Ainsworth, Manchester Metropolitan University Respondents Qing Gu, UCL Stephen Betts, Chief Executive of Learn Sheffield Mary Lowery, Head of School Improvement (Post-primary and Special Education, Locality East), Education Authority Northern Ireland In this latest edition of the What matters in education? panel discussion, Toby Greany and Steph Ainsworth will discuss their research into developing place-based approaches to school leadership development and tools that schools can use to improve teacher resilience and retention. An opportunity to reflect on the implications of their emerging findings for education policy and practice will follow, in conversation with Qing Gu, Professor of Leadership in Education at UCL; Stephen Betts, Chief Executive of Learn Sheffield and Mary Lowery Head of School Improvement (Post-primary and Special Education, Locality East), Education Authority Northern Ireland. Book your place A reminder that the ESRC Education Research Programme ECR Network cordially invites you for a special open session, featuring a panel of speakers on AI and the implications for teaching and learning in schools.",event


In [30]:
df.loc[df["output_type"] == "policy_engagement", ["text_x", "output_type", "link"]]

,text_x,output_type,link
110,"DfE - National Professional Qualifications framework review: call for evidence Government response Includes input to the study list from ERP PIs Toby Greany, Stephen Gorard and Rob Klassen listed.",policy_engagement,https://assets.publishing.service.gov.uk/media/686f911710d550c668de3d99/National_professional_qualifications_framework_review_call_for_evidence_response.pdf


In [31]:
# SAVE 

save_path = "/workspaces/erp_programme_updates/data/final_for_analysis.csv"
df.to_csv(save_path, index=False)

save_path

'/workspaces/erp_programme_updates/data/final_for_analysis.csv'